In [1]:
import pandas as pd
import glob

# Cargar todos los archivos de la variante BBR
csv_files = glob.glob("../data_processed/bbr/*.log.csv")
df = pd.concat([pd.read_csv(f, sep=';') for f in csv_files], ignore_index=True)

print(f"Cantidad de registros: {df.shape[0]}")
df.head()


Cantidad de registros: 179950


,"wscale,rto,rtt,mss,pmtu,rcvmss,advmss,cwnd,ssthresh,bytes_sent,bytes_retrans,bytes_acked,segs_out,segs_in,data_segs_out,lastrcv,delivered,rcv_space,rcv_ssthresh,timestamp,throughput,packets_sent,packets_acked,packets_lost,loss_rate"
0,"7.7,280,76.279,1448,1500,536,1448,14,52,26064,..."
1,"7.7,272,69.385,1448,1500,536,1448,4,52,37648,0..."
2,"7.7,240,39.996,1448,1500,536,1448,4,52,63712,0..."
3,"7.7,232,28.081,1448,1500,536,1448,14,52,101360..."
4,"7.7,248,45.456,1448,1500,536,1448,14,52,133216..."


In [ ]:
df['congestion_event'] = (df['cwnd'].diff().shift(-1) < -10).astype(int)
df['congestion_event'].value_counts()
